In [1]:
import torch
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

In [2]:
from msa_toolbox.utils.image.load_data_and_models import load_dataset
data = load_dataset("imagenet", "/mnt/disk1/sumdev/MSA_BTP/utils_train/data/Imagenet_full/Imagenet_2010_train/full_data", transform=True)
loader = DataLoader(data, batch_size=32, shuffle=True, num_workers=4)

In [3]:
for image, label, index in loader:
    print(image.shape, label.shape, index.shape)
    img = image[0]
    label = label[0]
    idx = index[0]
    print(img.shape, label, idx)
    path, label = data.samples[idx]
    print(path, label)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32]) torch.Size([32])
torch.Size([3, 224, 224]) tensor(484) tensor(622595)
/mnt/disk1/sumdev/MSA_BTP/utils_train/data/Imagenet_full/Imagenet_2010_train/full_data/n02981792.tar/n02981792_25938.JPEG 484


In [4]:
##################################################################################################
# In this section, we set the user authentication, user and app ID, model details, and the URL
# of the image we want as an input. Change these strings to run your own example.
#################################################################################################

# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = 'd041eff2b344468586a5d6666beaa38c'
# Specify the correct user_id/app_id     pairings
# Since you're making inferences outside your app's scope
USER_ID = 'clarifai'
APP_ID = 'main'
# Change these to whatever model and image URL you want to use
MODEL_ID = 'moderation-recognition'
MODEL_VERSION_ID = 'aa8be956dbaa4b7a858826a84253cab9'
IMAGE_URL = 'https://samples.clarifai.com/metro-north.jpg'


############################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
############################################################################

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)
metadata = (('authorization', 'Key ' + PAT),)
userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

image_path = '/mnt/disk1/sumdev/MSA_BTP/utils_train/data/Imagenet_full/Imagenet_2010_train/full_data/n01667114.tar/n01667114_14780.JPEG'
with open(image_path, "rb") as f:
    file_bytes = f.read()

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                id = '1',
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        # url=IMAGE_URL
                        base64=file_bytes,
                        allow_duplicate_url=True
                    )
                )
            ), 
            # resources_pb2.Input(
            #     id = '2',
            #     data=resources_pb2.Data(
            #         image=resources_pb2.Image(
            #             url=IMAGE_URL,
            #             allow_duplicate_url=True
            #         )
            #     )
            # )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

# Since we have one input, one output will exist here
class_to_idx = {}
for i in range(len(post_model_outputs_response.outputs)):
    output = post_model_outputs_response.outputs[i]
    print("Predicted concepts for input %d" % i)
    print(output.data.concepts)
    for concept in output.data.concepts:
        class_to_idx[concept.name] = len(class_to_idx)
        print("%s %.2f" % (concept.name, concept.value))

# Uncomment this line to print the full Response JSON
# print(output)

code: CONN_EXCEEDS_LIMITS
description: "Account limits exceeded"
details: "Request exceeds \"Pre-built classification model predict ops\" limit. Please contact support@clarifai.com or upgrade your plan."
req_id: "26bf7998063a16d829229e4e996f3b00"



Exception: Post model outputs failed, status: Account limits exceeded

In [ ]:
class_to_idx

{'safe': 0, 'drug': 1, 'suggestive': 2, 'gore': 3, 'explicit': 4}

In [ ]:
torch.argmax(torch.tensor([0.396, 0.604, 0.922])).item()

2

In [3]:
from PIL import Image
from transformers import pipeline

img = Image.open("/mnt/disk1/sumdev/MSA_BTP/utils_train/data/Imagenet_full/Imagenet_2010_train/full_data/n02981792.tar/n02981792_25938.JPEG")
# classifier = pipeline("image-classification", model="Falconsai/nsfw_image_detection")
classifier = pipeline("image-classification", model="apple/mobilevit-small")
classifier(img)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


[{'label': 'catamaran', 'score': 0.8138094544410706},
 {'label': 'trimaran', 'score': 0.10711844265460968},
 {'label': 'yawl', 'score': 0.0026023515965789557},
 {'label': 'schooner', 'score': 0.0007784378831274807},
 {'label': 'laptop, laptop computer', 'score': 0.0005865186103619635}]

In [4]:

import sys
old_stdout = sys.stdout

log_file = open("out/message.log","w")

sys.stdout = log_file

print("this will be written to message.log")

print("this will be written to message.log")

In [4]:
import json
import numpy as np
import boto3
runtime = boto3.Session().client(service_name='sagemaker-runtime', region_name='us-east-1')

# set the object categories array
object_categories = ['AnkleBoot','Bag','Coat','Dress','Pullover','Sandal','Shirt','Sneaker','TShirtTop','Trouser']

# Load the image bytes
img = open('/mnt/disk1/sumdev/MSA_BTP/utils_train/data/Imagenet_full/Imagenet_2010_train/full_data/n01667114.tar/n01667114_14780.JPEG', 'rb').read()
    
# Call your model for predicting which object appears in this image.
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-image', 
    Body=bytearray(img)
)
# read the prediction result and parse the json
result = response['Body'].read()
result = json.loads(result)

# which category has the highest confidence?
pred_label_id = np.argmax(result)

NameError: name 'endpoint_name' is not defined